# LPM-модель: подгонка модели и интерпретация коэффициентов

In [2]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.tools import add_constant
from statsmodels.iolib.summary2 import summary_col, summary_params

## Подгонка модели

## approve equation 1

Для датасета `loanapp` рассморим регрессию **approve на mortno, unem, dep, male, married, yjob, self**

Спецификация: 
$$
  approve=\beta_0+\beta_1mortno+\beta_2unem+\beta_3dep+\beta_4male+\beta_5married+\beta_6yjob+\beta_7self+u
$$

Альтернативная спецификация:
$$
  P(approve=1)=\beta_0+\beta_1mortno+\beta_2unem+\beta_3dep+\beta_4male+\beta_5married+\beta_6yjob+\beta_7self
$$

In [3]:
# подключим датасет loanapp по ссылке 
loanapp_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/econometrica/main/econometrica2/data-csv/loanapp.csv', na_values=(' ', '', '  '))
#подключим датасет loanapp из локального файла
#loanapp_df = pd.read_csv('loanapp.csv', na_values=(' ', '', '  '))

In [4]:
# посмотрим на первые пять строк датасета loanapp
loanapp_df.head(n=5)

,occ,loanamt,action,msa,suffolk,appinc,typur,unit,married,dep,...,approve,mortno,mortperf,mortlat1,mortlat2,chist,multi,loanprc,thick,white
0,1,89,1,1120,0,72,0,1.0,0.0,0.0,...,1,0,1,0,0,1,0.0,0.754237,0.0,1
1,1,128,3,1120,0,74,0,1.0,1.0,1.0,...,0,0,1,0,0,1,0.0,0.800000,1.0,1
2,1,128,1,1120,0,84,3,1.0,0.0,0.0,...,1,0,1,0,0,1,0.0,0.895105,1.0,1
3,1,66,1,1120,0,36,0,1.0,1.0,0.0,...,1,0,1,0,0,0,0.0,0.600000,0.0,1
4,1,120,1,1120,0,59,8,1.0,1.0,0.0,...,1,0,1,0,0,1,0.0,0.895522,0.0,1


In [5]:
# размеры датасета loanapp
loanapp_df.shape

(1989, 59)

In [6]:
# информация о датасете loanapp
loanapp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 59 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   occ       1989 non-null   int64  
 1   loanamt   1989 non-null   int64  
 2   action    1989 non-null   int64  
 3   msa       1989 non-null   int64  
 4   suffolk   1989 non-null   int64  
 5   appinc    1989 non-null   int64  
 6   typur     1989 non-null   int64  
 7   unit      1985 non-null   float64
 8   married   1986 non-null   float64
 9   dep       1986 non-null   float64
 10  emp       1989 non-null   int64  
 11  yjob      1989 non-null   int64  
 12  self      1989 non-null   int64  
 13  atotinc   1989 non-null   int64  
 14  cototinc  1989 non-null   float64
 15  hexp      1989 non-null   float64
 16  price     1989 non-null   float64
 17  other     1989 non-null   float64
 18  liq       1989 non-null   float64
 19  rep       1980 non-null   float64
 20  gdlin     1989 non-null   int6

In [7]:
# число пропусков по каждой переменной
loanapp_df.isna().sum()

occ           0
loanamt       0
action        0
msa           0
suffolk       0
appinc        0
typur         0
unit          4
married       3
dep           3
emp           0
yjob          0
self          0
atotinc       0
cototinc      0
hexp          0
price         0
other         0
liq           0
rep           9
gdlin         0
lines         0
mortg         0
cons          0
pubrec        0
hrat          0
obrat         0
fixadj        0
term          0
apr           0
prop          0
inss          0
inson         0
gift          0
cosign        0
unver         0
review        0
netw          0
unem          0
min30       183
bd            0
mi            0
old           0
vr            0
sch           0
black         0
hispan        0
male         15
reject        0
approve       0
mortno        0
mortperf      0
mortlat1      0
mortlat2      0
chist         0
multi         4
loanprc       0
thick         9
white         0
dtype: int64

### Поскольку регрессоры `dep, male` и `married` содержат пропуски (ячейки заполнены `NaN`), то получим сообщение об ошибке: `exog contains inf or nans`. 

#### `Необходимо эти пропуски удалить в ручную или автоматичеки (во втором способе)`

### 1 способ: зададим спецификацию модели через матрицы регрессионного дизайна 

[Документация по удалению строк со значением NaN](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)

In [9]:
# удалить все строки DataFrame, которые имеют значение NaN в столбцах mortno, unem, dep, male, married, yjob, self
loanapp_df = loanapp_df.dropna(subset=['mortno','unem','dep', 'male', 'married', 'yjob', 'self'])
#сбросим индекс DataFrame после удаления строк со значениями NaN
loanapp_df = loanapp_df.reset_index(drop=True)
loanapp_df

,occ,loanamt,action,msa,suffolk,appinc,typur,unit,married,dep,...,approve,mortno,mortperf,mortlat1,mortlat2,chist,multi,loanprc,thick,white
0,1,128,3,1120,0,74,0,1.0,1.0,1.0,...,0,0,1,0,0,1,0.0,0.800000,1.0,1
1,1,128,1,1120,0,84,3,1.0,0.0,0.0,...,1,0,1,0,0,1,0.0,0.895105,1.0,1
2,1,66,1,1120,0,36,0,1.0,1.0,0.0,...,1,0,1,0,0,0,0.0,0.600000,0.0,1
3,1,120,1,1120,0,59,8,1.0,1.0,0.0,...,1,0,1,0,0,1,0.0,0.895522,0.0,1
4,1,111,1,1120,0,63,9,1.0,0.0,0.0,...,1,0,1,0,0,0,0.0,0.804348,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1966,1,158,1,1120,0,96,0,1.0,1.0,0.0,...,1,0,1,0,0,1,0.0,0.897727,0.0,1
1967,1,35,1,1120,0,169,1,1.0,1.0,0.0,...,1,1,0,0,0,1,0.0,0.111111,0.0,1
1968,2,225,1,1120,0,49,0,2.0,1.0,0.0,...,1,0,1,0,0,1,1.0,1.000000,0.0,1
1969,1,98,1,1120,1,110,1,1.0,0.0,0.0,...,1,1,0,0,0,1,0.0,0.455814,0.0,1


In [10]:
# Зададим матрицы регрессионного дизайна
y = loanapp_df['approve'] # зависимая переменная
X = add_constant(loanapp_df[['mortno','unem','dep', 'male', 'married', 'yjob', 'self']]) # регрессоры с добавленным столбом единиц
X.head(n=7)

,const,mortno,unem,dep,male,married,yjob,self
0,1.0,0,3.2,1.0,1.0,1.0,0,0
1,1.0,0,3.9,0.0,1.0,0.0,1,0
2,1.0,0,3.1,0.0,1.0,1.0,0,1
3,1.0,0,4.3,0.0,1.0,1.0,0,0
4,1.0,0,3.2,0.0,1.0,0.0,0,0
5,1.0,0,3.2,0.0,1.0,0.0,1,0
6,1.0,0,3.2,0.0,1.0,0.0,1,0


In [11]:
# зададим спецификацию модели используя матрицы регрессионного дизайна 
mod_lpm = sm.OLS(endog=y, exog=X)
# подгонка модели с поправкой на гетероскедастичность
res_lpm_hc = mod_lpm.fit(cov_type='HC3')
# отчет
print(res_lpm_hc.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:                approve   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.017
No. Observations:                1971   F-statistic:                     5.849
Covariance Type:                  HC3   Prob (F-statistic):           9.77e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8642      0.023     37.135      0.000       0.819       0.910
mortno         0.0733      0.015      4.886      0.000       0.044       0.103
unem          -0.0064      0.004     -1.605      0.108      -0.014       0.001
dep           -0.0185      0.008     -2.429      0.015      -0.033      -0.004
male           0.0019      0.021      0.089      0.929      -0.040       0.044
married        0.0459      0.019      2.458      0.0

### 2 способ: зададим спецификацию модели через формулу

In [12]:
#зададим спецификацию модели через формулу
mod_lpm = smf.ols(formula='approve~mortno+unem+dep+male+married+yjob+self', data=loanapp_df)
# подгонка модели с поправкой на гетероскедастичность
res_lpm_hc = mod_lpm.fit(cov_type='HC3')
# отчет
print(res_lpm_hc.summary(slim=True))
# Альтеративно
# res_lpm_hc = sm.OLS.from_formula(formula='approve~mortno+unem+dep+male+married+yjob+self', data=loanapp_df)

                            OLS Regression Results                            
Dep. Variable:                approve   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.017
No. Observations:                1971   F-statistic:                     5.849
Covariance Type:                  HC3   Prob (F-statistic):           9.77e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8642      0.023     37.135      0.000       0.819       0.910
mortno         0.0733      0.015      4.886      0.000       0.044       0.103
unem          -0.0064      0.004     -1.605      0.108      -0.014       0.001
dep           -0.0185      0.008     -2.429      0.015      -0.033      -0.004
male           0.0019      0.021      0.089      0.929      -0.040       0.044
married        0.0459      0.019      2.458      0.0

## Интерпретация коэффициентов модели

Датасет `loanapp`

Основные переменные
	
- `approve` - бинарная, 1 если кредитная заявка одобрена 
- `appinc` - доход заявителя  (в \$1000)
- `mortno` -  бинарная, 1 если нет ипотечной кредитной истории 
- `unem` - уровень безработицы в отрасли в \% 
- `dep` - количество иждивенцев 
- `male` - гендерный фактор 
- `married` - семейный статус 
- `yjob` - стаж на текущей работе 
- `self` - бинарная, 1 если самозанятый 
	

In [13]:
# Коэфициенты модели с округлением до 3-х десятичных знаков
res_lpm_hc.params.round(3)

Intercept    0.864
mortno       0.073
unem        -0.006
dep         -0.018
male         0.002
married      0.046
yjob        -0.001
self        -0.036
dtype: float64

для регрессора `mortno`: разница **вероятностей** между уровнями 0 и 1 равна `0.073`

при увеличении регрессора `unem` на единицу **вероятность** `успеха` уменьшится на `0.006`

при увеличении регрессора `dep` на единицу **вероятность** `успеха` уменьшится на `0.018`

для регрессора `male`: разница **вероятностей** между уровнями 0 и 1 равна `0.002`

для регрессора `married`: разница **вероятностей** между уровнями 0 и 1 равна `0.046`

при увеличении регрессора `yjob` на единицу **вероятность** `успеха` уменьшится на `0.001`

для регрессора `self`: разница **вероятностей** между уровнями 0 и 1 равна `0.036`

## Сравнение моделей

In [15]:
# подгонка модели без поправки на гетероскедастичность 
res_lpm_ols = mod_lpm.fit(cov_type='nonrobust')
# Коэфициенты модели с округлением до 3-х десятичных знаков
res_lpm_ols.params.round(3)  

Intercept    0.864
mortno       0.073
unem        -0.006
dep         -0.018
male         0.002
married      0.046
yjob        -0.001
self        -0.036
dtype: float64

In [16]:
# Сравнение моделей
print(summary_col([res_lpm_hc, res_lpm_ols], model_names=['Robust', 'Non-robust'], stars=True))


                 Robust  Non-robust
-----------------------------------
Intercept      0.8642*** 0.8642*** 
               (0.0233)  (0.0219)  
mortno         0.0733*** 0.0733*** 
               (0.0150)  (0.0160)  
unem           -0.0064   -0.0064*  
               (0.0040)  (0.0035)  
dep            -0.0185** -0.0185** 
               (0.0076)  (0.0072)  
male           0.0019    0.0019    
               (0.0214)  (0.0203)  
married        0.0459**  0.0459*** 
               (0.0187)  (0.0176)  
yjob           -0.0007   -0.0007   
               (0.0062)  (0.0067)  
self           -0.0361   -0.0361   
               (0.0247)  (0.0223)  
R-squared      0.0203    0.0203    
R-squared Adj. 0.0168    0.0168    
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01
